### Betcity

In [97]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
from random import randint
import pandas as pd

executable_path = "C:\webdriver\chromedriver.exe"

try:
    driver = webdriver.Chrome(executable_path=executable_path)
except:
    driver = webdriver.Chrome()
    
url1 ='https://www.betcity.nl/sportsbook#sports-hub/volleyball/champions_league'
url = 'https://www.betcity.nl/sportsbook#event/1019027302'
driver.get(url)



C:\Users\root\AppData\Local\Temp\ipykernel_17872\3056313617.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=executable_path)


In [34]:
matches = driver.find_elements(By.XPATH, ".//li[@class='KambiBC-sandwich-filter__event-list-item KambiBC-sandwich-filter__event-list-competition-item']")
print(f'this is Matches: {matches}')

this is Matches: [<selenium.webdriver.remote.webelement.WebElement (session="d5f8960cc1dece2da2999d642b40c42f", element="f4987e23-364a-4ba1-aac8-44aa729557be")>]


In [35]:
matchLinks = []
for match in matches:
    print(f'his is Match: {match}')
    current_day = time.localtime()[6]
    try:
        match_day = match.find_element(By.XPATH, ".//span[@class='KambiBC-event-item__start-time--date']").text
        print(f'his is Match_day: {match_day}')

    except:
        continue
    '''
    print(f'start comparing')
    if match_day == "ma":
        match_day = 0
        print(f'his is Match_day: {match_day}')
    elif match_day == "di":
        match_day = 1
        print(f'his is Match_day: {match_day}')
    elif match_day == "wo":
        match_day = 2
        print(f'his is Match_day: {match_day}')
    elif match_day == "do":
        match_day = 3
        print(f'his is Match_day: {match_day}')
    elif match_day == "vr":
        match_day = 4
        print(f'his is Match_day: {match_day}')
    elif match_day == "za":
        print(f'his is Match_day: {match_day}')
        match_day = 5
    elif match_day == "zo":
        print(f'his is Match_day: {match_day}')
        match_day = 6
    else:
        print('continue')
        continue
              
    print(f'end comparing')

    date_diff = match_day - current_day
    if current_day != 6:
        if date_diff < 0 or date_diff > 1 :
            continue
    else:
        if match_day != 0 and match_day != 6:
            continue
    '''
    print(f'his is Match_day: {match_day}')

    link = match.find_element(By.XPATH, ".//a").get_attribute('href')
    print(f'his is Link: {link}')

    try:
        amount = int(match.find_element(By.XPATH, ".//div[@class='KambiBC-sandwich-filter_show-more-right-text']").text.split("B", 1)[0])
        print(f'his is Amount: {amount}')

    except:
        continue
    print('befor amount > 40')
    if link.find('live') == -1 and amount < 40:
        matchLinks.append(link)
        print(f'his is MatchLinks: {matchLinks}')

his is Match: <selenium.webdriver.remote.webelement.WebElement (session="d5f8960cc1dece2da2999d642b40c42f", element="f4987e23-364a-4ba1-aac8-44aa729557be")>
his is Match_day: 20 mei
his is Match_day: 20 mei
his is Link: https://www.betcity.nl/sportsbook#event/1019377350
his is Amount: 2
befor amount > 40
his is MatchLinks: ['https://www.betcity.nl/sportsbook#event/1019377350']


In [29]:
link.find('live')

-1

In [36]:
driver.get(link)

#### This for winnar/top2

In [98]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [112]:
# use beautiful soup to parse the html from selenium driver 
def volly_scrape(soup):
    teams = []
    winnar = []
    top2 = []
  
    t = soup.find_all(class_="KambiBC-outcomes-list__label")
    for i in range(len(t)):
        teams.append(t[i].text)
    
    numbers = soup.find_all(class_="OutcomeButton__Odds-sc-lxwzc0-6")
    for i in range(len(numbers)):
        if i < len(numbers)/2:
            winnar.append(float(numbers[i].text))
        else:
            top2.append(float(numbers[i].text))
    return(teams,  winnar, top2)

In [113]:
volly_scrape(soup)

['Sir Safety Susa Perugia', 'Itas Trentino', 'Gas Sales Bluenergy Piacenza', 'Modena Volley', 'Cucine Lube Civitanova', 'Allianz Milano', 'Vero Volley Monza', 'WithU Verona', 'Top Volley Cisterna', 'Emma Villas Aubay Siena'] [1.25, 7.85, 10.0, 12.0, 14.0, 61.0, 61.0, 61.0, 501.0, 9001.0] [1.08, 2.3, 2.9, 4.0, 4.5, 13.0, 13.0, 13.0, 251.0, 6001.0]


In [111]:
teams

[<span class="KambiBC-outcomes-list__label"><span>Sir Safety Susa Perugia</span></span>,
 <span class="KambiBC-outcomes-list__label"><span>Itas Trentino</span></span>,
 <span class="KambiBC-outcomes-list__label"><span>Gas Sales Bluenergy Piacenza</span></span>,
 <span class="KambiBC-outcomes-list__label"><span>Modena Volley</span></span>,
 <span class="KambiBC-outcomes-list__label"><span>Cucine Lube Civitanova</span></span>,
 <span class="KambiBC-outcomes-list__label"><span>Allianz Milano</span></span>,
 <span class="KambiBC-outcomes-list__label"><span>Vero Volley Monza</span></span>,
 <span class="KambiBC-outcomes-list__label"><span>WithU Verona</span></span>,
 <span class="KambiBC-outcomes-list__label"><span>Top Volley Cisterna</span></span>,
 <span class="KambiBC-outcomes-list__label"><span>Emma Villas Aubay Siena</span></span>]

In [102]:
teams = soup.find_all(class_="KambiBC-outcomes-list__label")

In [104]:
teams[0].text

'Sir Safety Susa Perugia'

Sir Safety Susa Perugia
Itas Trentino
Gas Sales Bluenergy Piacenza
Modena Volley
Cucine Lube Civitanova
Allianz Milano
Vero Volley Monza
WithU Verona
Top Volley Cisterna
Emma Villas Aubay Siena


In [115]:
%pwd

'c:\\Users\\root\\Desktop\\notebook\\1-Freelancer work\\BetcityScrape'

In [117]:
import pickle

In [118]:
# open pickle file
with open('df_betcity_volly_test', 'rb') as f:
    df = pickle.load(f)


In [119]:
df.head()

,Teams,Winnar,Top2
0,"[Sir Safety Conad Perugia, Itas Trentino, Jast...","[1.5, 6.75, 8.2, 10.0, 13.0, 26.0, 81.0, 101.0]","[1.3, 4.6, 2.15, 2.6, 8.25, 6.5, 16.0, 51.0]"
0,"[Tours VB, Chaumont, St Nazaire, Nantes Rezé M...","[1.65, 8.0, 8.0, 12.0, 12.0, 14.0, 17.0, 17.0,...","[1.33, 2.75, 2.75, 5.0, 5.0, 5.5, 6.5, 6.5, 26..."
0,"[Nantes Volley (D), Volero Le Cannet (D), Aspt...","[3.35, 5.25, 5.75, 6.75, 6.75, 10.0, 11.0, 31....","[1.8, 2.8, 3.1, 3.4, 3.4, 4.2, 4.5, 13.0, 18.0..."
0,"[Sir Safety Susa Perugia, Itas Trentino, Gas S...","[1.45, 3.9, 8.5, 18.0]","[1.18, 1.22, 4.25, 3.8]"
0,"[Imoco Volley Conegliano (D), Pallavolo Scandi...","[1.5, 5.0, 6.0, 7.5, 65.0, 71.0, 91.0, 151.0, ...","[1.18, 1.72, 2.25, 2.55, 23.0, 26.0, 31.0, 41...."


## LivesScoreBet

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import chromedriver_autoinstaller

import requests  # 2
import json  # 3

from pyvirtualdisplay import Display
display = Display(visible=0, size=(800, 800))
display.start()

# Check if the current version of chromedriver exists
chromedriver_autoinstaller.install()
# and if it doesn't exist, download it automatically,
# then add chromedriver to path

chrome_options = webdriver.ChromeOptions()
# Add your options as needed
options = [
    # Define window size here
    "--window-size=1200,1200",
    "--ignore-certificate-errors"

    # "--headless",
    # "--disable-gpu",
    # "--window-size=1920,1200",
    # "--ignore-certificate-errors",
    # "--disable-extensions",
    # "--no-sandbox",
    # "--disable-dev-shm-usage",
    # '--remote-debugging-port=9222'
]

for option in options:
    chrome_options.add_argument(option)


driver = webdriver.Chrome(options=chrome_options)

#driver.get("https://www.livescorebet.com/") #ok

url = 'https://www.livescorebet.com/nl-nl/sports/volleybal/SBTC1_19'
driver.get(url)
print(driver.title)
sleep(5000)

WebDriverException: Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: chrome=110.0.5481.100)
Stacktrace:
#0 0x563120f02b03 <unknown>
#1 0x563120cd2dad <unknown>
#2 0x563120cbee4c <unknown>
#3 0x563120cbe2f1 <unknown>
#4 0x563120cbd37b <unknown>
#5 0x563120cbd1c4 <unknown>
#6 0x563120cbbca2 <unknown>
#7 0x563120cbc442 <unknown>
#8 0x563120cc979f <unknown>
#9 0x563120cca3f2 <unknown>
#10 0x563120cda7d0 <unknown>
#11 0x563120cdea70 <unknown>
#12 0x563120cbc903 <unknown>
#13 0x563120cda40c <unknown>
#14 0x563120d4b2fa <unknown>
#15 0x563120d32df3 <unknown>
#16 0x563120d02900 <unknown>
#17 0x563120d03af8 <unknown>
#18 0x563120f570ce <unknown>
#19 0x563120f5b090 <unknown>
#20 0x563120f3b590 <unknown>
#21 0x563120f5c31d <unknown>
#22 0x563120f2c928 <unknown>
#23 0x563120f813a8 <unknown>
#24 0x563120f81536 <unknown>
#25 0x563120f9b3f7 <unknown>
#26 0x7f613a184609 start_thread
